In [6]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time



In [7]:

api_key = 'B6MOR128S6URKDZ0'


In [8]:
ts = TimeSeries(key=api_key, output_format = 'pandas')

data, meta_data = ts.get_intraday(symbol='MSFT', interval = '1min', outputsize = 'full')
print(data)

                      1. open   2. high    3. low  4. close  5. volume
date                                                                  
2019-12-09 16:00:00  151.4000  151.4000  151.2300  151.3600   345096.0
2019-12-09 15:59:00  151.3550  151.4000  151.3550  151.4000   104520.0
2019-12-09 15:58:00  151.3000  151.3500  151.2700  151.3500    83872.0
2019-12-09 15:57:00  151.3200  151.3300  151.2800  151.2905    58178.0
2019-12-09 15:56:00  151.3900  151.3900  151.2900  151.3100    93941.0
...                       ...       ...       ...       ...        ...
2019-12-03 09:35:00  147.6182  147.6182  147.6182  147.6182   123548.0
2019-12-03 09:34:00  147.1299  147.5703  147.0265  147.4353   131089.0
2019-12-03 09:33:00  147.0200  147.3300  147.0200  147.1100   197631.0
2019-12-03 09:32:00  147.3729  147.4860  146.7654  147.0129   407257.0
2019-12-03 09:31:00  147.1200  147.6300  147.0600  147.3800  1219138.0

[1931 rows x 5 columns]


In [10]:
i = 1
# while i==1:
#     data, meta_data = ts.get_intraday(symbol='MSFT', interval = '1min', outputsize = 'full')
#     data.to_excel("output.xlsx")
#     time.sleep(60)
    

In [13]:
close_data = data['4. close']

In [14]:
percentage_change = close_data.pct_change()
print(percentage_change)

date
2019-12-09 16:00:00         NaN
2019-12-09 15:59:00    0.000264
2019-12-09 15:58:00   -0.000330
2019-12-09 15:57:00   -0.000393
2019-12-09 15:56:00    0.000129
                         ...   
2019-12-03 09:35:00   -0.000056
2019-12-03 09:34:00   -0.001239
2019-12-03 09:33:00   -0.002206
2019-12-03 09:32:00   -0.000660
2019-12-03 09:31:00    0.002497
Name: 4. close, Length: 1931, dtype: float64


In [16]:
last_change = percentage_change[-1]

In [27]:
if abs(last_change) > 0.0004:
    print(last_change)
    print("Change seen")
else: 
    print("No change")
    

0.00249705978182857
Change seen
